In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamd

In [4]:
df = pd.read_csv('Songs/songs0.csv')

# remove "spotify:track:", "spotify:artist:", and "spotify:album:" from uri
df["track_uri"] = df["track_uri"].str[14:]
df["artist_uri"] = df["artist_uri"].str[15:]
df["album_uri"] = df["album_uri"].str[14:]

In [126]:
df.head()

,pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,Toxic,0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,Crazy In Love,25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,Rock Your Body,6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [127]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

b'{"access_token":"BQAu3DvXX3COYcOb4I7SVBFosFx20Nz3GqKTWebKMY8ycYaA2vCv6O93jo3IZRE5XyFBTpV4w3JlkcTkrBY","token_type":"Bearer","expires_in":3600,"scope":""}'


In [136]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

[{'acousticness': 0.0311,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'danceability': 0.904,
  'duration_ms': 226864,
  'energy': 0.813,
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'instrumentalness': 0.00697,
  'key': 4,
  'liveness': 0.0471,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'tempo': 125.461,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'type': 'audio_features',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'valence': 0.81}]

In [8]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

In [9]:
tracks = songs_in_playlist(df, 20)
playlist_means(tracks)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.027000,https://api.spotify.com/v1/audio-analysis/1iDP...,0.914,125217,0.415,1iDPB2WNsGEWuKycQyDGn6,0.000000,1,0.1140,-7.219,1,0.3220,137.974,4,https://api.spotify.com/v1/tracks/1iDPB2WNsGEW...,audio_features,spotify:track:1iDPB2WNsGEWuKycQyDGn6,0.4960
1,0.000404,https://api.spotify.com/v1/audio-analysis/0AIC...,0.659,143969,0.614,0AICBlLzFCTpUqmAbtzB2z,0.000000,9,0.1230,-7.320,0,0.0541,119.405,4,https://api.spotify.com/v1/tracks/0AICBlLzFCTp...,audio_features,spotify:track:0AICBlLzFCTpUqmAbtzB2z,0.2600
2,0.003650,https://api.spotify.com/v1/audio-analysis/1c5L...,0.934,192000,0.407,1c5Lbn9mYwNYbnNiizw6D0,0.000000,0,0.1100,-8.058,1,0.3450,140.024,4,https://api.spotify.com/v1/tracks/1c5Lbn9mYwNY...,audio_features,spotify:track:1c5Lbn9mYwNYbnNiizw6D0,0.0793
3,0.363000,https://api.spotify.com/v1/audio-analysis/4s2B...,0.708,182267,0.612,4s2BWgnSQ9NZiOQM4PP4HB,0.000000,6,0.2040,-8.939,0,0.3020,94.274,4,https://api.spotify.com/v1/tracks/4s2BWgnSQ9NZ...,audio_features,spotify:track:4s2BWgnSQ9NZiOQM4PP4HB,0.3410
4,0.002640,https://api.spotify.com/v1/audio-analysis/7GX5...,0.732,182707,0.750,7GX5flRQZVHRAGd6B4TmDO,0.000000,11,0.1090,-6.366,0,0.2310,155.096,4,https://api.spotify.com/v1/tracks/7GX5flRQZVHR...,audio_features,spotify:track:7GX5flRQZVHRAGd6B4TmDO,0.4010
5,0.057900,https://api.spotify.com/v1/audio-analysis/3uFm...,0.917,122996,0.624,3uFmweGlFudKNiX13XFwFm,0.000000,4,0.0714,-6.728,1,0.1470,124.984,4,https://api.spotify.com/v1/tracks/3uFmweGlFudK...,audio_features,spotify:track:3uFmweGlFudKNiX13XFwFm,0.3430
6,0.015900,https://api.spotify.com/v1/audio-analysis/0Fcz...,0.955,115316,0.773,0FczZscGB1PynHk6zwNO69,0.000172,1,0.0809,-3.062,1,0.2830,135.039,4,https://api.spotify.com/v1/tracks/0FczZscGB1Py...,audio_features,spotify:track:0FczZscGB1PynHk6zwNO69,0.2380
7,0.066700,https://api.spotify.com/v1/audio-analysis/4Cb4...,0.821,191990,0.713,4Cb4Ehfj2S8UhC5JKqmJ6m,0.000000,0,0.0923,-4.814,0,0.2430,160.988,4,https://api.spotify.com/v1/tracks/4Cb4Ehfj2S8U...,audio_features,spotify:track:4Cb4Ehfj2S8UhC5JKqmJ6m,0.6140
8,0.140000,https://api.spotify.com/v1/audio-analysis/7zez...,0.939,145171,0.617,7zez4ZwqfSqD6fPQgcnqwu,0.000000,7,0.1120,-5.933,0,0.2980,131.949,4,https://api.spotify.com/v1/tracks/7zez4ZwqfSqD...,audio_features,spotify:track:7zez4ZwqfSqD6fPQgcnqwu,0.6800
9,0.006130,https://api.spotify.com/v1/audio-analysis/7MCs...,0.691,201721,0.715,7MCs283jdifycrZoMPPynX,0.000000,8,0.0368,-7.215,1,0.0589,77.223,4,https://api.spotify.com/v1/tracks/7MCs283jdify...,audio_features,spotify:track:7MCs283jdifycrZoMPPynX,0.4650


---
**Note**

We could construct a model which determines if a user will like a certain playlist based on comparing their playlists and liked playlists to the audio features of our playlist. Train model on users playlists and likes, classify playlist as 0 or 1 if the user will like the playlist.

I believe our final model will look something like this: https://developer.spotify.com/console/get-recommendations/

In [96]:
songs = pd.DataFrame()
ranges1 = np.arange(0,10, 1)
for n in ranges1:
    songs = songs.append(playlist_means(songs_in_playlist(df, n)))
    time.sleep(1)
songs.to_pickle("songs_1_10.pkl")


In [98]:
songs = pd.read_pickle("songs_1_10.pkl")
songs.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.03110,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,125.461,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,0.02490,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,143.040,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,0.00238,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,99.259,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,0.20200,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,100.972,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,0.05610,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,94.759,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [100]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    for col in cols: 
        x_scaled = MinMaxScaler().fit_transform(df_old[col].reshape(-1,1))
        df_new[col] = x_scaled
    return(df_new)
        
songs2 = scaler(cols, songs, songs2)
songs2.head()

/Users/williamdrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.031436,https://api.spotify.com/v1/audio-analysis/0UaM...,0.917143,226864,0.822126,0UaMYEvWZi0ZqiDOoHU3YI,0.007033,4,0.022881,-7.105,0,0.202823,0.472799,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,1,0.025167,https://api.spotify.com/v1/audio-analysis/6I9V...,0.771582,198800,0.847831,6I9VzXrHxO9rA9A5euc8Ak,0.025227,5,0.242561,-3.914,0,0.188291,0.581461,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,2,0.002397,https://api.spotify.com/v1/audio-analysis/0WqI...,0.648416,235933,0.765577,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.037196,-6.583,0,0.387586,0.310835,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,3,0.204239,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.902586,267267,0.720337,1AWQoqb9bSvzTjaLralEkT,0.000236,4,0.028517,-6.055,0,0.242267,0.321424,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,4,0.056715,https://api.spotify.com/v1/audio-analysis/1lzr...,0.860038,227600,0.609295,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.322588,-4.596,1,0.099647,0.283019,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [101]:
song1 = songs2.iloc[np.where(songs.id == '0WqIKmW4BTrj3eJFmnCKMv')]

In [128]:
def distance(idkey, songs):
    song = songs.iloc[np.where(songs.id == idkey)]
#     comp = []
#     for col in cols:
#         comp.append(np.subtract(songs[col], song[col])**2)
    dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                 + (np.subtract(songs.danceability, song.danceability))**2
                 + (np.subtract(songs.energy, song.energy))**2
                 + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                 + (np.subtract(songs.liveness, song.liveness))**2
                 + (np.subtract(songs.speechiness, song.speechiness))**2
                 + (np.subtract(songs.tempo, song.tempo))**2)
        
    return dist

In [129]:
top10_dist = distance('0WqIKmW4BTrj3eJFmnCKMv', songs2).sort_values()[0:10]

In [144]:
top10_dist.index

Int64Index([2, 374, 104, 45, 113, 370, 98, 257, 407, 21], dtype='int64')

In [131]:
top10 = songs2.iloc[top10_dist.index]

In [145]:
hrefs = top10['id']
hrefs

2      0WqIKmW4BTrj3eJFmnCKMv
374    7fYbFYt7X4FZvuJJC90EX0
104    4b1sQpvL7QVgamRZ74F1oA
45     1X5WGCrUMuwRFuYU1eAo2I
113    2MSjhkw47sGrjbZhColyVx
370    6FE2iI43OZnszFLuLtvvmg
98     2qWgqPdW1OiAP8KSBH1b93
257    6Xio661Y2iL7SDKFFOG9cv
407    3cdcT7I1xnQqQ84zMCtrKU
21     0uqPG793dkDDN7sCUJJIVC
Name: id, dtype: object

In [153]:
for ref in top10['id']:
    track = sp.track(ref)
    print(track['name'])
    print('by')
    for artist in track['artists']:
        print(artist['name'])
    print('-----------------------------------')

Crazy In Love (feat. Jay-Z)
by
Beyoncé
JAY Z
-----------------------------------
Problem
by
Ariana Grande
Iggy Azalea
-----------------------------------
Don't Recall
by
KARD
-----------------------------------
Livin on Sunday
by
Cris Cab
-----------------------------------
Aquaman
by
Jay Park
-----------------------------------
Classic
by
MKTO
-----------------------------------
FXXK WIT US
by
LEE HI
Dok2
-----------------------------------
Who Got Da Props
by
Black Moon
-----------------------------------
Hyperreal
by
Flume
KUČKA
-----------------------------------
Where Is The Love?
by
The Black Eyed Peas
-----------------------------------
